# Day 20
https://adventofcode.com/2020/day/20

In [661]:
import math

f = open("Day 20.txt", "r")
batch = f.read().split('\n\n') #Split by new lines
batch = [x.split('\n') for x in batch]
#print(len(batch))

tiles = dict()
for x in batch:
    tiles[int(x[0][5:-1])] = x[1:]

#Edges
edges = dict()

for i in tiles:
    #print(i)
    tmp = []
    tmp.append(tiles[i][0])
    tmp.append(''.join([x[-1] for x in tiles[i]]))
    tmp.append(tiles[i][-1])

    tmp.append(''.join([x[0] for x in tiles[i]]))
    edges[i] = tmp
    #break
    
num_tiles = len(tiles)
l = int(math.sqrt(num_tiles))

print(num_tiles, l)
#print(edges)

144 12


## Part 1

In [662]:
def reverse(string):
    return ''.join(list(reversed(list(string))))

def rotate_once(e_l):
    return tuple([e_l[1], reverse(e_l[2]), e_l[3], reverse(e_l[0])])

def rotations(k, e_l):
    
    all_edges = dict()
    #edges_set = set()
    letters = [chr(ord('@')+i+1) for i in range(26)]
    i = 0
    
    #regular edge - rotate clockwise
    all_edges[str(k)+letters[i]] = e_l
    i = i + 1
        
    for x in range(1,4):
        all_edges[str(k)+letters[i]] = rotate_once(all_edges[str(k)+letters[i-1]])
        i = i + 1
        
    #print(all_edges)
    
    #flip vertically down
    all_edges[str(k)+letters[i]] = [e_l[2], reverse(e_l[1]), e_l[0], reverse(e_l[3])]
    i = i + 1
    
    for x in range(1,4):
        all_edges[str(k)+letters[i]] = rotate_once(all_edges[str(k)+letters[i-1]])
        i = i + 1
    
    return all_edges

In [671]:
all_edges = dict()

for k in edges:
    all_edges.update(rotations(k,edges[k]))

#print(len(all_edges))
#for x in all_edges: print(x, all_edges[x])

In [672]:
nesw = [(-1,0), (0,1), (1,0), (0,-1)]
next = [2, 3, 0, 1]
keys = [k for k in all_edges]

index = 12

placement = dict()
placement[keys[index]] = (0,0)

keys_to_check = []
keys_to_check.append(keys[index])

curr_key = keys[index]
curr_edge = all_edges[curr_key]

while (len(placement) < num_tiles):

    if len(keys_to_check) == 0: break
    curr_key = keys_to_check.pop(0) #get the first key to check
    curr_edge = all_edges[curr_key]
    x, y = placement[curr_key]

    #print("Round", i, "Coordinates", (x, y), curr_key, curr_edge)

    #look at each direction
    for n in range(4):
        neighbor = (x+nesw[n][0], y+nesw[n][1])

        key_nums = set([k[:4] for k in placement])
        check_set = [k for k in keys if k[:4] not in key_nums]

        check_edges = [(c,all_edges[c][next[n]]) for c in check_set]
        #print(check_edges)

        match_edges = [i for i, x in enumerate(check_edges) if x[1]==curr_edge[n]]

        if len(match_edges) == 1:
            match_key = check_edges[match_edges[0]][0]
            keys_to_check.append(match_key)
            placement[match_key] = neighbor

#print(placement)  
#print(len(placement))
X = set([placement[x][0] for x in placement])
Y = set([placement[x][1] for x in placement])
corners = [(min(X), min(Y)), (min(X), max(Y)), (max(X), min(Y)), (max(X), max(Y))]
#print(corners)
#print([int(x[:4]) for x in placement if placement[x] in corners])
print(math.prod([int(x[:4]) for x in placement if placement[x] in corners]))


placement2 = dict((v,k) for k,v in placement.items()) #flip dictionary
#print(placement)
print([placement2[c] for c in corners])

111936085519519
['3593D', '3517A', '2797E', '3167D']


## Part 2

In [665]:
def rotateTile(tile, orientation):
    
    if orientation == 'A': return tile
            
    num = ord(orientation) - 65
    #print(num)
    
    #flip first
    if num >= 4:
        new_tile = [tile[i] for i in range(len(tile)-1,-1,-1)]
        num = num - 4
    else:
        new_tile = tile
    
    #print(new_tile)
    num = (4 - num) % 4
    for i in range(num):
        new_tile = list(zip(*new_tile[::-1]))
    
    return [''.join(x) for x in new_tile]

def getImage(key):
    
    key_int = int(key[:4])
    key_rot = key[-1]
    
    return rotateTile(tiles[key_int], key_rot)

In [668]:
placement2 = dict((v,k) for k,v in placement.items()) #flip dictionary

grid = dict()
big_tile = []

for x_i, x in enumerate(sorted(X)):
    for y_i, y in enumerate(sorted(Y)):
        #print((x_i,y_i),(x,y), placement2[(x,y)])
        new_tile = getImage(placement2[(x,y)])
        #for t in new_tile: print(t)
        #print('\n')
    
        new_tile = [t[1:-1] for t in new_tile[1:-1]]
        if x_i not in grid:
            grid[x_i] = new_tile
        else:
            grid[x_i] = [x+y for x,y in zip(grid[x_i], new_tile)]
    
for x in grid:
    for y in grid[x]:
        big_tile.append(y)

#print(len(big_tile), len(big_tile[0]))
#for x in big_tile: print(x)
#for x in rotateTile(big_tile, 'E'): print(x)

In [670]:
def findMonster(tile, pattern):
    
    pattern_i = []
    for p in pattern:
        pattern_i.append([i for i,v in enumerate(p) if v == '#'])
        
    #print(pattern_i)
    #pattern_i = [i for i in pattern]
    
    l = len(pattern)
    w = len(pattern[0])
    
    loop_i = len(tile) - l + 1
    loop_j = len(tile[0]) - w + 1
    
    #print(l,w,loop_i, loop_j)
    
    count = 0
    for i in range(loop_i):
        for j in range(loop_j):
            sub_tile = [t[j:j+w] for t in tile[i:i+l]]
            #for x in sub_tile: print(x)
            
            isValid = True
            for k in range(l):
                #print(pattern_i[k])
                #print(sub_tile[k])
                
                #print([sub_tile[k][i] for i in pattern_i[k]]
                if [sub_tile[k][i] for i in pattern_i[k]].count('.') > 0:
                    isValid = False
                    break
            
            count += isValid
            #break
        #break
    
    return count
    

    
monster = '''                  # 
#    ##    ##    ###
 #  #  #  #  #  #   '''
num_monster = monster.count('#')
monster = monster.split('\n')

#print(num_monster)
#print(monster)

#for x in big_tile: print(x)
#print("\n")

for i in range(8):
    letter = chr(ord('@')+i+1)
    count = findMonster(rotateTile(big_tile, letter), monster)
    if count > 0: break

#Calculate #'s in tile
sum([i.count('#') for i in big_tile]) - count * num_monster

1792